### Part 1: Equation fitting

#### Step 0: Preparation

- Import Libraries

In [1]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import pandas as pd
import numpy as np


- สร้าง function สำหรับ print สมการ (อ่านข้ามส่วนนี้ไปได้)

- เตรียมข้อมูล

In [3]:
# ดึงข้อมูลจากไฟล์ csv
df = pd.read_csv("input.csv")
df.head()

,Y,X1,X2,X3
0,222000.0,1,6.3,2016
1,177000.0,1,5.5,2016
2,177777.0,1,5.5,2016
3,199999.0,1,5.5,2016
4,199999.0,1,5.5,2016


In [11]:
# แบ่ง dataframe เป็นตัวแปร X และ y
# ในตัวอย่างนี้จะใช้แค่ X1 ในการสร้างสมการ
X = df[[df.columns[1]]] #X input
y = df[df.columns[0]]

In [12]:
X

,X1
0,1
1,1
2,1
3,1
4,1
5,1
6,1
7,1
8,1
9,1


In [6]:
y

0       222000.00
1       177000.00
2       177777.00
3       199999.00
4       199999.00
5       199999.00
6       199999.00
7       185555.00
8       187555.00
9       189999.00
10      129999.00
11      154999.00
12       99999.00
13       99999.00
14       99999.00
15      100000.00
16      102999.00
17      103999.00
18      103999.00
19      195000.00
20      195000.00
21      139000.00
22      124000.00
23      109000.00
24       66346.00
25       67622.55
26       71138.00
27       72000.00
28       80999.00
29       81500.00
          ...    
3973      3950.00
3974      3100.00
3975      1999.00
3976      2650.00
3977      2200.00
3978      1100.00
3979      1852.50
3980      8000.00
3981      1750.00
3982      4300.00
3983      1850.00
3984      2200.00
3985      6000.00
3986      3200.00
3987      2650.00
3988      1300.00
3989      1899.00
3990      2500.00
3991      3700.00
3992      2400.00
3993      1950.00
3994      1296.75
3995       600.00
3996      7777.00
3997      

#### Step 1: สร้างสมการ 1 ตัวแปร

เริ่มจากสมการเส้นตรงธรรมดากันก่อน
- Linear fitting degree 1 (1 feature)

In [13]:
# Linear fitting: y = ax + b
lg = LinearRegression()
lg.fit(X, y)


Equation: y = -1.18E+02*X+3.90E+04
R-Square = 0.224



In [14]:
# Slope (m) from equation: (y = mx + c)
lg.coef_

array([-118.33491101])

In [16]:
# Interception (c)  from equation: (y = mx + c)
lg.intercept_

38957.29770006296

In [18]:
# lg score (R-square)
lg.score(X,y)

0.22376321884491931

## แสดงผลออกมา

เราสร้าง function เสริมออกมา เพื่อทำให้การ print ดูง่ายขึ้น

* เพื่อรับค่า feature name (Xn), ค่า coef (m), interception (c), และ Polynomial degree แล้วสร้างเป็นสมการออกมา
* return เป็น string ที่ทำให้อ่านง่ายขึ้น

In [20]:
from decimal import Decimal

# Function: Generate polynomial equation
def Poly_equation(feature, coef, intercept, power):
    '''
    Return fitted polynomial equation as a string
    
    feature: list of feature in dataset
    power: degree of each feature in each term (can get from poly.powers_)
    '''
    poly_string = ""
    
    for i in range(len(coef)): # create polynomial term
        
        #Coefficients
        if i == 0:
            term_string = "y = %.2E" % Decimal(coef[i])
        elif coef[i] >= 0: # add + sign in front of coef
            term_string = "+%.2E" % Decimal(coef[i])
        else:
            term_string = "%.2E" % Decimal(coef[i])
        
        #Powers
        feature_order = 0
        for power_iter in power[i]: # power for each feature
            if power_iter == 1 : #degree of that feature = 1
                term_string += '*' + str(feature[feature_order])
            elif power_iter > 1 : #degree of that feature > 1
                term_string += '*' + str(feature[feature_order]) + '^' + str(power_iter)
            feature_order += 1
        poly_string += term_string
    
    #Add intercept
    if intercept >= 0:
        poly_string += "+"
    poly_string += "%.2E" % Decimal(intercept)
    
    return poly_string

# Function: Generate log equation
def Log_equation(feature,a,b):
    # Desired equation: y = bln(x)+a
    log_string = "y = %.2E*ln(%s)" % (Decimal(b),feature)
    if a >= 0:
        log_string += "+"
    log_string += "%.2E" % Decimal(a)
    return log_string

# Function: Generate exponential equation
def Exp_equation(feature,d,e):
    # Desired equation: y = exp(e)*exp(d*x)
    exp_string = "y = %.2E*exp(%.2E*%s)" % (Decimal(np.exp(e)),Decimal(d),feature)
    return exp_string

# Function: Generate power equation
def Power_equation(feature,f,g):
    # Desired equation: y = exp(g)*x^f
    pow_string = "y = %.2E*%s^(%.2E)" % (Decimal(np.exp(g)),feature,Decimal(f))
    return pow_string

In [21]:
# print สมการและค่า R-square
# from function: Poly_equation
print("Equation: " + Poly_equation('X1', lg.coef_, lg.intercept_, [[1]]))
print("R-Square = "+ str(np.round(lg.score(X,y),3))+"\n")

Equation: y = -1.18E+02*X+3.90E+04
R-Square = 0.224



ลองมาสร้างสมการพหุนามกันต่อ
- Linear fitting degree 3 (1 feature)  <br>
ขั้นแรกคือการนำตัวแปรที่มีไปสร้างเป็นตัวแปรพหุนามตัวแปรใหม่ ตั้งชื่อว่า X_

In [22]:
nPoly = 3 #กำหนดให้เป็นสมการดีกรี 3
# create polynomial feature
poly = PolynomialFeatures(degree=nPoly, include_bias=None)
# transform features to polynomial terms
X_ = poly.fit_transform(X) 
print("Polynomial degree: "+str(nPoly))
print("Degree for each feature:\n" + str(poly.powers_))

Polynomial degree: 3
Degree for each feature:
[[1]
 [2]
 [3]]


In [27]:
X[-5:]

,X1
3998,10
3999,300
4000,261
4001,440
4002,150


In [28]:
X_[-5:,:]

array([[1.0000000e+01, 1.0000000e+02, 1.0000000e+03],
       [3.0000000e+02, 9.0000000e+04, 2.7000000e+07],
       [2.6100000e+02, 6.8121000e+04, 1.7779581e+07],
       [4.4000000e+02, 1.9360000e+05, 8.5184000e+07],
       [1.5000000e+02, 2.2500000e+04, 3.3750000e+06]])

นั่นคือจะได้มา 3 พจน์ได้แก่ x, x^2 และ x^3 <br>
หลังจากนั้น ลองสร้างสมการเส้นตรงจากตัวแปรใหม่ (x, x^2, x^3)

In [29]:
# Linear fitting: y = ax^3 + bx^2 + cx + d
lg = LinearRegression()
lg.fit(X_, y)
# print สมการและค่า R-square
print("Equation: " + Poly_equation('X1', lg.coef_, lg.intercept_, poly.powers_))
print("R-Square = "+ str(np.round(lg.score(X_,y),3))+"\n")

Equation: y = -3.85E+02*X+9.05E-01*X^2-5.66E-04*X^3+5.46E+04
R-Square = 0.326



สมการแบบอื่นก็มี เช่น Expo, log, power <br>
สำหรับสมการในรูปแบบ Exponential, Natural log และ Power จะใช้ฟังก์ชัน polyfit ของ numpy ในการหาค่าคงที่

<b> Exponential fitting </b> <br>

In [ ]:
#Exponential fitting: y = a*exp(b*X)
d,e = np.polyfit(X.loc[:,'X1'], np.log(y), 1) # input for polyfit must be 1D numpy array
print(Exp_equation('X1',d,e))

<b> Natural log fitting </b>

In [ ]:
# Natural log fitting: y = a+bln(X)
b,a = np.polyfit(X.loc[:,'X1'], y, 1)
print(Log_equation('X1',a,b))

<b> Power fitting </b>

In [ ]:
#Power fitting: y = aX^b
f,g = np.polyfit(X.loc[:,'X1'],np.log(y),1)
print(Power_equation('X1',f,g))

#### Step 2: สร้างสมการ 2 ตัวแปร

<b> Linear fitting degree 3 (2 features) </b> <br>
วิธีทำไม่ต่างจากการสร้างสมการ 1 ตัวแปรมากนัก

In [30]:
# ในตัวอย่างนี้จะใช้ X1 และ X2 ในการสร้างสมการ
XX = df.drop(df.columns[[0,3]], axis=1)

In [31]:
nPoly = 3
# create polynomial feature
poly = PolynomialFeatures(degree=nPoly, include_bias=None)
# transform features to polynomial terms
XX_ = poly.fit_transform(XX)
print("Polynomial degree: "+str(nPoly))
print("Degree for each feature(X1,X2):\n" + str(poly.powers_))

Polynomial degree: 3
Degree for each feature(X1,X2):
[[1 0]
 [0 1]
 [2 0]
 [1 1]
 [0 2]
 [3 0]
 [2 1]
 [1 2]
 [0 3]]


นั่นคือจะได้มา 9 พจน์ได้แก่ x1, x2, x1^2, x1\*x2, x2^2, x1^3, x1^2\*x2, x1\*x2^2 และ x2^3 <br>
หลังจากนั้น ลองสร้างสมการเส้นตรงจากตัวแปรใหม่

In [32]:
# Linear fitting
lg = LinearRegression()
lg.fit(XX_, y)
# print สมการและค่า R-square
print("Equation: " + Poly_equation(['X1','X2'], lg.coef_, lg.intercept_, poly.powers_))
print("\nR-Square = "+ str(np.round(lg.score(XX_,y),3))+"\n")

Equation: y = 6.78E+01*X1+2.39E+04*X2+1.72E-01*X1^2-1.63E+02*X1*X2+2.06E+03*X2^2-6.79E-04*X1^3+3.06E-01*X1^2*X2-5.72E+00*X1*X2^2-2.14E+02*X2^3-1.61E+04

R-Square = 0.639

